# <i>   Miniproyecto 2

### Alejandro Tolosa
---

### 2.1	Perceptrón multicapa

#### 2.1.1	Modificaciones de la arquitectura

El objetivo de esta sección de la actividad es evaluar diversas arquitecturas para el perceptrón multicapa y seleccionar el modelo que obtenga el mejor rendimiento en términos de accuracy. Para ello, se propone modificar los siguientes elementos: el número de neuronas por capa, la cantidad de capas ocultas y el tipo de función de activación utilizada.

##### <i> 1) Probaremos el código orginal del modelo de Perceptron Multicapa para tener un parámetro de comparación

In [1]:
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms


########################################################################
##########################        DATOS       ##########################
########################################################################

transform = transforms.Compose([
    transforms.ToTensor()  # Convierte la imagen de un formato PIL o numpy.ndarray a un tensor.
])
mnist_dataset = datasets.MNIST(
    root='./data', train=True, transform=transform, download=True
)
data_loader = DataLoader(
    mnist_dataset, batch_size=16, shuffle=True
)
images, labels = next(iter(data_loader))



########################################################################
#######################      ARQUITECTURA       ########################
########################################################################


class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 200)
        self.fc2 = nn.Linear(200, 256)
        self.fc3 = nn.Linear(256, 10)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Aplanamos las imágenes (de 28x28 a 1D)
        x = self.relu(self.fc1(x))  # Aplicamos la primera capa y ReLU
        x = self.dropout(x)         # Aplicamos Dropout
        x = self.relu(self.fc2(x))  # Aplicamos la segunda capa y ReLU
        x = self.dropout(x)         
        x = self.fc3(x)             
        return x
    


########################################################################
######################      ENTRENAMIENTO       ########################
########################################################################

batch_size = 64
learning_rate = 0.001
epochs = 10

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_dataset = datasets.MNIST(
    root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(
    root='./data', train=False, transform=transform, download=True)
train_loader = DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=False)

model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Época [{epoch+1}/{epochs}], Pérdida: {loss.item():.4f}")


########################################################################
##################      EVALUACIÓN DEL MODELO       ####################
########################################################################


model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy en el conjunto de prueba: {accuracy:.2f}%")

Época [1/10], Pérdida: 0.1195
Época [2/10], Pérdida: 0.2527
Época [3/10], Pérdida: 0.3766
Época [4/10], Pérdida: 0.0501
Época [5/10], Pérdida: 0.0718
Época [6/10], Pérdida: 0.0213
Época [7/10], Pérdida: 0.0998
Época [8/10], Pérdida: 0.2025
Época [9/10], Pérdida: 0.0451
Época [10/10], Pérdida: 0.1758
Accuracy en el conjunto de prueba: 97.42%


##### <i> 2) Experimento 1A: Modificamos la arquitectura del modelo reduciendo la cantidad de neuronas a 128 en la capa de entrada, en la capa oculta se pasa de 128 a 64 neuronas

In [2]:
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms


########################################################################
##########################        DATOS       ##########################
########################################################################

transform = transforms.Compose([
    transforms.ToTensor()  # Convierte la imagen de un formato PIL o numpy.ndarray a un tensor.
])
mnist_dataset = datasets.MNIST(
    root='./data', train=True, transform=transform, download=True
)
data_loader = DataLoader(
    mnist_dataset, batch_size=16, shuffle=True
)
images, labels = next(iter(data_loader))



########################################################################
#######################      ARQUITECTURA       ########################
########################################################################

class Experimento_1A(nn.Module):
    def __init__(self):
        super(Experimento_1A, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

########################################################################
######################      ENTRENAMIENTO       ########################
########################################################################

batch_size = 64
learning_rate = 0.001
epochs = 10

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_dataset = datasets.MNIST(
    root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(
    root='./data', train=False, transform=transform, download=True)
train_loader = DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=False)

model = Experimento_1A()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Época [{epoch+1}/{epochs}], Pérdida: {loss.item():.4f}")


########################################################################
##################      EVALUACIÓN DEL MODELO       ####################
########################################################################


model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy en el conjunto de prueba: {accuracy:.2f}%")

Época [1/10], Pérdida: 0.4061
Época [2/10], Pérdida: 0.2350
Época [3/10], Pérdida: 0.5393
Época [4/10], Pérdida: 0.2173
Época [5/10], Pérdida: 0.1800
Época [6/10], Pérdida: 0.4296
Época [7/10], Pérdida: 0.0945
Época [8/10], Pérdida: 0.0591
Época [9/10], Pérdida: 0.1964
Época [10/10], Pérdida: 0.1723
Accuracy en el conjunto de prueba: 97.10%


##### <i> 3) Experimento 1B: Modificamos la arquitectura del modelo aumentando la cantidad de neuronas a 400 en la capa de entrada, en la capa oculta se pasa de 400 a 512 neuronas

In [3]:
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms


########################################################################
##########################        DATOS       ##########################
########################################################################

transform = transforms.Compose([
    transforms.ToTensor()  # Convierte la imagen de un formato PIL o numpy.ndarray a un tensor.
])
mnist_dataset = datasets.MNIST(
    root='./data', train=True, transform=transform, download=True
)
data_loader = DataLoader(
    mnist_dataset, batch_size=16, shuffle=True
)
images, labels = next(iter(data_loader))



########################################################################
#######################      ARQUITECTURA       ########################
########################################################################


class Experimento_1B(nn.Module):
    def __init__(self):
        super(Experimento_1B, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 400)
        self.fc2 = nn.Linear(400, 512)
        self.fc3 = nn.Linear(512, 10)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x


########################################################################
######################      ENTRENAMIENTO       ########################
########################################################################

batch_size = 64
learning_rate = 0.001
epochs = 10

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_dataset = datasets.MNIST(
    root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(
    root='./data', train=False, transform=transform, download=True)
train_loader = DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=False)

model = Experimento_1B()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Época [{epoch+1}/{epochs}], Pérdida: {loss.item():.4f}")


########################################################################
##################      EVALUACIÓN DEL MODELO       ####################
########################################################################


model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy en el conjunto de prueba: {accuracy:.2f}%")

Época [1/10], Pérdida: 0.2067
Época [2/10], Pérdida: 0.2464
Época [3/10], Pérdida: 0.0590
Época [4/10], Pérdida: 0.0451
Época [5/10], Pérdida: 0.0494
Época [6/10], Pérdida: 0.1392
Época [7/10], Pérdida: 0.3267
Época [8/10], Pérdida: 0.0060
Época [9/10], Pérdida: 0.0068
Época [10/10], Pérdida: 0.0128
Accuracy en el conjunto de prueba: 97.70%


##### <i> 4) Experimento 1C: Modificamos la arquitectura del modelo aumentando la cantidad de neuronas a 784 en la capa de entrada, en la capa oculta se pasa de 784 a 1000 neuronas

In [4]:
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

########################################################################
##########################        DATOS       ##########################
########################################################################

transform = transforms.Compose([
    transforms.ToTensor()  # Convierte la imagen de un formato PIL o numpy.ndarray a un tensor.
])
mnist_dataset = datasets.MNIST(
    root='./data', train=True, transform=transform, download=True
)
data_loader = DataLoader(
    mnist_dataset, batch_size=16, shuffle=True
)
images, labels = next(iter(data_loader))



########################################################################
#######################      ARQUITECTURA       ########################
########################################################################


class Experimento_1C(nn.Module):
    def __init__(self):
        super(Experimento_1C, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 784)
        self.fc2 = nn.Linear(784, 1000)
        self.fc3 = nn.Linear(1000, 10)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x


########################################################################
######################      ENTRENAMIENTO       ########################
########################################################################

batch_size = 64
learning_rate = 0.001
epochs = 10

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_dataset = datasets.MNIST(
    root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(
    root='./data', train=False, transform=transform, download=True)
train_loader = DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=False)

model = Experimento_1C()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Época [{epoch+1}/{epochs}], Pérdida: {loss.item():.4f}")


########################################################################
##################      EVALUACIÓN DEL MODELO       ####################
########################################################################


model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy en el conjunto de prueba: {accuracy:.2f}%")

Época [1/10], Pérdida: 0.2230
Época [2/10], Pérdida: 0.1562
Época [3/10], Pérdida: 0.1538
Época [4/10], Pérdida: 0.1560
Época [5/10], Pérdida: 0.2300
Época [6/10], Pérdida: 0.0410
Época [7/10], Pérdida: 0.0705
Época [8/10], Pérdida: 0.0023
Época [9/10], Pérdida: 0.1548
Época [10/10], Pérdida: 0.0079
Accuracy en el conjunto de prueba: 97.41%


Elabore un informe detallado de los experimentos realizados, incluyendo las configuraciones probadas, los resultados obtenidos y las conclusiones que justifiquen la selección del modelo final.

<i> En este experimento modificamos en la arquitectura la cantidad de neuronas en la capa de entrada y en la capa oculta. En la arquitectura del modelo original cuenta con 200 neuronas de entrada aumentando a 256 en la salida; el experimento 1A dismunuímos la cantidad de neuronas a 128 de entrada y a 64 de salida; el experimento 1B aumentamos la cantidad de neuronas de entrada a 400 y a 512 de salida y por último el experimento 1C aumentamos la cantidad de neuronas a 784 de entrada y 1000 de salida.

Los resultados del expperimento fueron los siguientes:

In [5]:
import pandas as pd

data = {
    'Época': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'Arq Original': [0.1195, 0.2527, 0.3766, 0.0501, 0.0718, 0.0213, 0.0998, 0.2025, 0.0451, 0.1758],
    'Exp 1A (128/64)': [0.4061, 0.2350, 0.5393, 0.2173, 0.1800, 0.4296, 0.0945, 0.0591, 0.1964, 0.1723],
    'Exp 1B (400/512)': [0.2067, 0.2464, 0.0590, 0.0451, 0.0494, 0.1392, 0.3267, 0.0060, 0.0068, 0.0128],
    'Exp 1C (784/1000)': [0.2230, 0.1562, 0.1538, 0.1560, 0.2300, 0.0410, 0.0705, 0.0023, 0.1548, 0.0079]
}

df = pd.DataFrame(data)

print(df)


   Época  Arq Original  Exp 1A (128/64)  Exp 1B (400/512)  Exp 1C (784/1000)
0      1        0.1195           0.4061            0.2067             0.2230
1      2        0.2527           0.2350            0.2464             0.1562
2      3        0.3766           0.5393            0.0590             0.1538
3      4        0.0501           0.2173            0.0451             0.1560
4      5        0.0718           0.1800            0.0494             0.2300
5      6        0.0213           0.4296            0.1392             0.0410
6      7        0.0998           0.0945            0.3267             0.0705
7      8        0.2025           0.0591            0.0060             0.0023
8      9        0.0451           0.1964            0.0068             0.1548
9     10        0.1758           0.1723            0.0128             0.0079


<i> La tabla anteriomente mostrada representa las pérdida del modelo durante la época del entrenamiento, podemos ver que en la arquitectura original comienza con el mejor rendimiento, pero se mantiene moderada. El experimento 1A tiene mayor variabilidad en la pérdida, lo que podemos interpretar que la cantidad reducida de neuronas tiene menor capacidad para caputurar los datos de entrenamiento. El experimento 1B es el que en promedio tiene mejor rendimiento destacándose en las últimas tes épocas. El experimento 1C que es el que cuenta con mayor cantidad de neuronas, mantiene una cantidad de pérdida baja pero aparentemente no mejora con respecto al experimento 1B, por lo que se puede concluir que se está sobreajustando.

In [6]:
import pandas as pd

data = {
    'Experimento': [
        'Arquitectura Original (200/256)',
        'Experimento 1A (128/64)',
        'Experimento 1B (400/512)',
        'Experimento 1C (784/1000)'
    ],
    'Accuracy (%)': [97.42, 97.10, 97.70, 97.41],
    'Tiempo de Ejecución': ['1 min 26.7 sec', '1 min 23.9 sec', '1 min 47.1 sec', '2 min 18.2 sec']
}

df = pd.DataFrame(data)

print(df)


                       Experimento  Accuracy (%) Tiempo de Ejecución
0  Arquitectura Original (200/256)         97.42      1 min 26.7 sec
1          Experimento 1A (128/64)         97.10      1 min 23.9 sec
2         Experimento 1B (400/512)         97.70      1 min 47.1 sec
3        Experimento 1C (784/1000)         97.41      2 min 18.2 sec


<i> Como conclusión de esta primera parte, podemos señalar lo siguiente:
- Al reducir la cantidad de neuronas representado por el experimento 1A, tiene menos tiempo de ejecución 2,8 segundos menos que la arquitectura original, pero pierde la capacidad de procesar datos en un 0,32% menos que la arquitectura original, por lo que tiene mayor probabilidad de error.
- Al aumentar la cantidad de neuronas a 400/512, aumenta el tiempo de procesamiento en 20,4 segundos y tiene un un 0,28 puntos de mejor rendimiento
- Al aumentar mucho la cantidad de neuronas (experimento 1C), reduce el rendiento en 0,01%, pero aumenta el tiempo de ejeución en 51,5 segundos lo que implica que este modelo de arquitectura se sobreajusta y gasta mas tiempo.
- En términos de rendimiento el experimento 1B es el que mejor desempeño tiene, pero la arquitectura original se destaca por ser la segunda con mejor rendimiento y por menos de 3 segundos mas de ejecución que el experimento mas rápido, por lo que en términos de la relacion tiempo y calidad es mejor la arquitectura original.

---

#### 2.1.2	Modificaciones del entrenamiento

El objetivo de esta sección de la actividad es evaluar distintas estrategias de entrenamiento y seleccionar la mejor alternativa en función de su accuracy. Para ello, se propone modificar los siguientes aspectos: el algoritmo de optimización, la tasa de aprendizaje (learning rate), el tamaño del lote (batch size) y el número de épocas.

##### <i> 1) Probaremos el código orginal del modelo de Perceptron Multicapa para tener un parámetro de comparación

In [17]:
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms


########################################################################
##########################        DATOS       ##########################
########################################################################

transform = transforms.Compose([
    transforms.ToTensor()  # Convierte la imagen de un formato PIL o numpy.ndarray a un tensor.
])
mnist_dataset = datasets.MNIST(
    root='./data', train=True, transform=transform, download=True
)
data_loader = DataLoader(
    mnist_dataset, batch_size=16, shuffle=True
)
images, labels = next(iter(data_loader))



########################################################################
#######################      ARQUITECTURA       ########################
########################################################################


class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 200)
        self.fc2 = nn.Linear(200, 256)
        self.fc3 = nn.Linear(256, 10)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Aplanamos las imágenes (de 28x28 a 1D)
        x = self.relu(self.fc1(x))  # Aplicamos la primera capa y ReLU
        x = self.dropout(x)         # Aplicamos Dropout
        x = self.relu(self.fc2(x))  # Aplicamos la segunda capa y ReLU
        x = self.dropout(x)         
        x = self.fc3(x)             
        return x
    


########################################################################
######################      ENTRENAMIENTO       ########################
########################################################################

batch_size = 64
learning_rate = 0.001
epochs = 10

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_dataset = datasets.MNIST(
    root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(
    root='./data', train=False, transform=transform, download=True)
train_loader = DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=False)

model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Época [{epoch+1}/{epochs}], Pérdida: {loss.item():.4f}")


########################################################################
##################      EVALUACIÓN DEL MODELO       ####################
########################################################################


model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy en el conjunto de prueba: {accuracy:.2f}%")

Época [1/10], Pérdida: 0.2386
Época [2/10], Pérdida: 0.1160
Época [3/10], Pérdida: 0.2797
Época [4/10], Pérdida: 0.1868
Época [5/10], Pérdida: 0.0235
Época [6/10], Pérdida: 0.0166
Época [7/10], Pérdida: 0.0283
Época [8/10], Pérdida: 0.0519
Época [9/10], Pérdida: 0.2089
Época [10/10], Pérdida: 0.1948
Accuracy en el conjunto de prueba: 97.57%


##### <i> 2) Experimento 2A: Modificaremos los hiperparámetros del modelo de entrenamiento, disminuimos el batch size  a la mitad y mantenemos el resto de los hiperparámetros

In [18]:
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms


########################################################################
##########################        DATOS       ##########################
########################################################################

transform = transforms.Compose([
    transforms.ToTensor()  # Convierte la imagen de un formato PIL o numpy.ndarray a un tensor.
])
mnist_dataset = datasets.MNIST(
    root='./data', train=True, transform=transform, download=True
)
data_loader = DataLoader(
    mnist_dataset, batch_size=16, shuffle=True
)
images, labels = next(iter(data_loader))



########################################################################
#######################      ARQUITECTURA       ########################
########################################################################


class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 200)
        self.fc2 = nn.Linear(200, 256)
        self.fc3 = nn.Linear(256, 10)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Aplanamos las imágenes (de 28x28 a 1D)
        x = self.relu(self.fc1(x))  # Aplicamos la primera capa y ReLU
        x = self.dropout(x)         # Aplicamos Dropout
        x = self.relu(self.fc2(x))  # Aplicamos la segunda capa y ReLU
        x = self.dropout(x)         
        x = self.fc3(x)             
        return x
    


########################################################################
######################      ENTRENAMIENTO       ########################
########################################################################

BS_Experimento_2A = 32
LR_Experimento_2A = 0.001
E_Experimento_2A = 10

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_dataset = datasets.MNIST(
    root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(
    root='./data', train=False, transform=transform, download=True)
train_loader = DataLoader(
    dataset=train_dataset, batch_size=BS_Experimento_2A, shuffle=True)
test_loader = DataLoader(
    dataset=test_dataset, batch_size=BS_Experimento_2A, shuffle=False)

model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR_Experimento_2A)

for epoch in range(E_Experimento_2A):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Época [{epoch+1}/{E_Experimento_2A}], Pérdida: {loss.item():.4f}")


########################################################################
##################      EVALUACIÓN DEL MODELO       ####################
########################################################################


model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy en el conjunto de prueba: {accuracy:.2f}%")

Época [1/10], Pérdida: 0.2043
Época [2/10], Pérdida: 0.0841
Época [3/10], Pérdida: 0.0871
Época [4/10], Pérdida: 0.6107
Época [5/10], Pérdida: 0.2925
Época [6/10], Pérdida: 0.1648
Época [7/10], Pérdida: 0.4410
Época [8/10], Pérdida: 0.1740
Época [9/10], Pérdida: 0.0310
Época [10/10], Pérdida: 0.3183
Accuracy en el conjunto de prueba: 97.49%


##### <i> 3) Experimento 2B: Modificaremos los hiperparámetros del modelo de entrenamiento, aumentaremos al doble el batch size con respecto al original y mantenemos el resto de los hiperparámetros

In [19]:
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms


########################################################################
##########################        DATOS       ##########################
########################################################################

transform = transforms.Compose([
    transforms.ToTensor()  # Convierte la imagen de un formato PIL o numpy.ndarray a un tensor.
])
mnist_dataset = datasets.MNIST(
    root='./data', train=True, transform=transform, download=True
)
data_loader = DataLoader(
    mnist_dataset, batch_size=16, shuffle=True
)
images, labels = next(iter(data_loader))



########################################################################
#######################      ARQUITECTURA       ########################
########################################################################


class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 200)
        self.fc2 = nn.Linear(200, 256)
        self.fc3 = nn.Linear(256, 10)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Aplanamos las imágenes (de 28x28 a 1D)
        x = self.relu(self.fc1(x))  # Aplicamos la primera capa y ReLU
        x = self.dropout(x)         # Aplicamos Dropout
        x = self.relu(self.fc2(x))  # Aplicamos la segunda capa y ReLU
        x = self.dropout(x)         
        x = self.fc3(x)             
        return x
    


########################################################################
######################      ENTRENAMIENTO       ########################
########################################################################

BS_Experimento_2B = 128
LR_Experimento_2B = 0.001
E_Experimento_2B = 10

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_dataset = datasets.MNIST(
    root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(
    root='./data', train=False, transform=transform, download=True)
train_loader = DataLoader(
    dataset=train_dataset, batch_size=BS_Experimento_2B, shuffle=True)
test_loader = DataLoader(
    dataset=test_dataset, batch_size=BS_Experimento_2B, shuffle=False)

model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR_Experimento_2B)

for epoch in range(E_Experimento_2B):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Época [{epoch+1}/{E_Experimento_2B}], Pérdida: {loss.item():.4f}")


########################################################################
##################      EVALUACIÓN DEL MODELO       ####################
########################################################################


model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy en el conjunto de prueba: {accuracy:.2f}%")

Época [1/10], Pérdida: 0.3723
Época [2/10], Pérdida: 0.2989
Época [3/10], Pérdida: 0.0330
Época [4/10], Pérdida: 0.1279
Época [5/10], Pérdida: 0.1575
Época [6/10], Pérdida: 0.0899
Época [7/10], Pérdida: 0.1164
Época [8/10], Pérdida: 0.0675
Época [9/10], Pérdida: 0.0401
Época [10/10], Pérdida: 0.0411
Accuracy en el conjunto de prueba: 97.63%


##### 4) Experimento 2C: Modificaremos los hiperparámetros del modelo de entrenamiento, aumentaremos la tasa de aprendizaje a 0,005 y mantenemos el resto de los hiperparámetros

In [21]:
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms


########################################################################
##########################        DATOS       ##########################
########################################################################

transform = transforms.Compose([
    transforms.ToTensor()  # Convierte la imagen de un formato PIL o numpy.ndarray a un tensor.
])
mnist_dataset = datasets.MNIST(
    root='./data', train=True, transform=transform, download=True
)
data_loader = DataLoader(
    mnist_dataset, batch_size=16, shuffle=True
)
images, labels = next(iter(data_loader))



########################################################################
#######################      ARQUITECTURA       ########################
########################################################################


class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 200)
        self.fc2 = nn.Linear(200, 256)
        self.fc3 = nn.Linear(256, 10)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Aplanamos las imágenes (de 28x28 a 1D)
        x = self.relu(self.fc1(x))  # Aplicamos la primera capa y ReLU
        x = self.dropout(x)         # Aplicamos Dropout
        x = self.relu(self.fc2(x))  # Aplicamos la segunda capa y ReLU
        x = self.dropout(x)         
        x = self.fc3(x)             
        return x
    


########################################################################
######################      ENTRENAMIENTO       ########################
########################################################################

BS_Experimento_2C = 64
LR_Experimento_2C = 0.005
E_Experimento_2C = 10

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_dataset = datasets.MNIST(
    root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(
    root='./data', train=False, transform=transform, download=True)
train_loader = DataLoader(
    dataset=train_dataset, batch_size=BS_Experimento_2C, shuffle=True)
test_loader = DataLoader(
    dataset=test_dataset, batch_size=BS_Experimento_2C, shuffle=False)

model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR_Experimento_2C)

for epoch in range(E_Experimento_2C):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Época [{epoch+1}/{E_Experimento_2C}], Pérdida: {loss.item():.4f}")


########################################################################
##################      EVALUACIÓN DEL MODELO       ####################
########################################################################


model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy en el conjunto de prueba: {accuracy:.2f}%")

Época [1/10], Pérdida: 0.6212
Época [2/10], Pérdida: 0.7610
Época [3/10], Pérdida: 0.1028
Época [4/10], Pérdida: 0.3005
Época [5/10], Pérdida: 0.1551
Época [6/10], Pérdida: 0.2499
Época [7/10], Pérdida: 0.3746
Época [8/10], Pérdida: 0.2485
Época [9/10], Pérdida: 0.3358
Época [10/10], Pérdida: 0.2431
Accuracy en el conjunto de prueba: 94.94%


##### <i> 5) Experimento 2D: Modificaremos los hiperparámetros del modelo de entrenamiento, disminuiremos la tasa de aprendizaje a 0,0005 y mantenemos el resto de los hiperparámetros

In [22]:
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms


########################################################################
##########################        DATOS       ##########################
########################################################################

transform = transforms.Compose([
    transforms.ToTensor()  # Convierte la imagen de un formato PIL o numpy.ndarray a un tensor.
])
mnist_dataset = datasets.MNIST(
    root='./data', train=True, transform=transform, download=True
)
data_loader = DataLoader(
    mnist_dataset, batch_size=16, shuffle=True
)
images, labels = next(iter(data_loader))



########################################################################
#######################      ARQUITECTURA       ########################
########################################################################


class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 200)
        self.fc2 = nn.Linear(200, 256)
        self.fc3 = nn.Linear(256, 10)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Aplanamos las imágenes (de 28x28 a 1D)
        x = self.relu(self.fc1(x))  # Aplicamos la primera capa y ReLU
        x = self.dropout(x)         # Aplicamos Dropout
        x = self.relu(self.fc2(x))  # Aplicamos la segunda capa y ReLU
        x = self.dropout(x)         
        x = self.fc3(x)             
        return x
    


########################################################################
######################      ENTRENAMIENTO       ########################
########################################################################

BS_Experimento_2D = 64
LR_Experimento_2D = 0.0005
E_Experimento_2D = 10

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_dataset = datasets.MNIST(
    root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(
    root='./data', train=False, transform=transform, download=True)
train_loader = DataLoader(
    dataset=train_dataset, batch_size=BS_Experimento_2D, shuffle=True)
test_loader = DataLoader(
    dataset=test_dataset, batch_size=BS_Experimento_2D, shuffle=False)

model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR_Experimento_2D)

for epoch in range(E_Experimento_2D):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Época [{epoch+1}/{E_Experimento_2D}], Pérdida: {loss.item():.4f}")


########################################################################
##################      EVALUACIÓN DEL MODELO       ####################
########################################################################


model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy en el conjunto de prueba: {accuracy:.2f}%")

Época [1/10], Pérdida: 0.3044
Época [2/10], Pérdida: 0.3030
Época [3/10], Pérdida: 0.1127
Época [4/10], Pérdida: 0.1753
Época [5/10], Pérdida: 0.0362
Época [6/10], Pérdida: 0.0142
Época [7/10], Pérdida: 0.0509
Época [8/10], Pérdida: 0.0125
Época [9/10], Pérdida: 0.0948
Época [10/10], Pérdida: 0.2154
Accuracy en el conjunto de prueba: 97.77%


##### <i> 6) Experimento 2E: Modificaremos los hiperparámetros del modelo de entrenamiento, aumentaremos la época a 15 y mantenemos el resto de los hiperparámetros

In [23]:
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms


########################################################################
##########################        DATOS       ##########################
########################################################################

transform = transforms.Compose([
    transforms.ToTensor()  # Convierte la imagen de un formato PIL o numpy.ndarray a un tensor.
])
mnist_dataset = datasets.MNIST(
    root='./data', train=True, transform=transform, download=True
)
data_loader = DataLoader(
    mnist_dataset, batch_size=16, shuffle=True
)
images, labels = next(iter(data_loader))



########################################################################
#######################      ARQUITECTURA       ########################
########################################################################


class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 200)
        self.fc2 = nn.Linear(200, 256)
        self.fc3 = nn.Linear(256, 10)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Aplanamos las imágenes (de 28x28 a 1D)
        x = self.relu(self.fc1(x))  # Aplicamos la primera capa y ReLU
        x = self.dropout(x)         # Aplicamos Dropout
        x = self.relu(self.fc2(x))  # Aplicamos la segunda capa y ReLU
        x = self.dropout(x)         
        x = self.fc3(x)             
        return x
    


########################################################################
######################      ENTRENAMIENTO       ########################
########################################################################

BS_Experimento_2E = 64
LR_Experimento_2E = 0.001
E_Experimento_2E = 15

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_dataset = datasets.MNIST(
    root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(
    root='./data', train=False, transform=transform, download=True)
train_loader = DataLoader(
    dataset=train_dataset, batch_size=BS_Experimento_2E, shuffle=True)
test_loader = DataLoader(
    dataset=test_dataset, batch_size=BS_Experimento_2E, shuffle=False)

model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR_Experimento_2E)

for epoch in range(E_Experimento_2E):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Época [{epoch+1}/{E_Experimento_2E}], Pérdida: {loss.item():.4f}")


########################################################################
##################      EVALUACIÓN DEL MODELO       ####################
########################################################################


model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy en el conjunto de prueba: {accuracy:.2f}%")

Época [1/15], Pérdida: 0.1026
Época [2/15], Pérdida: 0.1162
Época [3/15], Pérdida: 0.1466
Época [4/15], Pérdida: 0.1791
Época [5/15], Pérdida: 0.0314
Época [6/15], Pérdida: 0.0845
Época [7/15], Pérdida: 0.0321
Época [8/15], Pérdida: 0.2514
Época [9/15], Pérdida: 0.0879
Época [10/15], Pérdida: 0.0309
Época [11/15], Pérdida: 0.1307
Época [12/15], Pérdida: 0.2140
Época [13/15], Pérdida: 0.1333
Época [14/15], Pérdida: 0.0033
Época [15/15], Pérdida: 0.0569
Accuracy en el conjunto de prueba: 97.57%


##### <i> 7) Experimento 2F: Modificaremos los hiperparámetros del modelo de entrenamiento, disminuiremos la época a 5 y mantenemos el resto de los hiperparámetros

In [24]:
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms


########################################################################
##########################        DATOS       ##########################
########################################################################

transform = transforms.Compose([
    transforms.ToTensor()  # Convierte la imagen de un formato PIL o numpy.ndarray a un tensor.
])
mnist_dataset = datasets.MNIST(
    root='./data', train=True, transform=transform, download=True
)
data_loader = DataLoader(
    mnist_dataset, batch_size=16, shuffle=True
)
images, labels = next(iter(data_loader))



########################################################################
#######################      ARQUITECTURA       ########################
########################################################################


class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 200)
        self.fc2 = nn.Linear(200, 256)
        self.fc3 = nn.Linear(256, 10)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Aplanamos las imágenes (de 28x28 a 1D)
        x = self.relu(self.fc1(x))  # Aplicamos la primera capa y ReLU
        x = self.dropout(x)         # Aplicamos Dropout
        x = self.relu(self.fc2(x))  # Aplicamos la segunda capa y ReLU
        x = self.dropout(x)         
        x = self.fc3(x)             
        return x
    


########################################################################
######################      ENTRENAMIENTO       ########################
########################################################################

BS_Experimento_2F = 64
LR_Experimento_2F = 0.001
E_Experimento_2F = 5

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_dataset = datasets.MNIST(
    root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(
    root='./data', train=False, transform=transform, download=True)
train_loader = DataLoader(
    dataset=train_dataset, batch_size=BS_Experimento_2F, shuffle=True)
test_loader = DataLoader(
    dataset=test_dataset, batch_size=BS_Experimento_2F, shuffle=False)

model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR_Experimento_2F)

for epoch in range(E_Experimento_2F):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Época [{epoch+1}/{E_Experimento_2F}], Pérdida: {loss.item():.4f}")


########################################################################
##################      EVALUACIÓN DEL MODELO       ####################
########################################################################


model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy en el conjunto de prueba: {accuracy:.2f}%")

Época [1/5], Pérdida: 0.1449
Época [2/5], Pérdida: 0.0943
Época [3/5], Pérdida: 0.2317
Época [4/5], Pérdida: 0.1525
Época [5/5], Pérdida: 0.0174
Accuracy en el conjunto de prueba: 97.03%


Elabore un informe detallado de los experimentos realizados, incluyendo las configuraciones evaluadas, los resultados obtenidos y las conclusiones que respalden la selección de la estrategia final.

<i> Realizamos dos experimentos por cada hiperparámetro del modelo de entrenamiento uno aumentando y el otro disminuyendo el hiperparámetro respecto al modelo de entrenamiento original y esto fueron los resultados:

In [27]:
import pandas as pd

data = {
    'Experimento': [
        'Entrenamiento Original',
        'Exp 2A (Batch Size:32)',
        'Exp 2B (Batch Size:128)',
        'Exp 2C (Learning Rate: 0.005)',
        'Exp 2D (Learning Rate: 0.0005)',
        'Exp 2E (Epochs:15)',
        'Exp 2F (Epochs:5)'
    ],
    'Accuracy (%)': [
        97.57,
        97.49,
        97.63,
        94.94,
        97.77,
        97.57,
        97.03
    ],
    'Tiempo de Ejecución': [
        '1 min 28.6 seg',
        '1 min 59.6 seg',
        '1 min 21.5 seg',
        '1 min 30.6 seg',
        '1 min 29.2 seg',
        '2 min 18.2 seg',
        '0 min 46.3 seg'
    ]
}

df = pd.DataFrame(data)

print(df)


                      Experimento  Accuracy (%) Tiempo de Ejecución
0          Entrenamiento Original         97.57      1 min 28.6 seg
1          Exp 2A (Batch Size:32)         97.49      1 min 59.6 seg
2         Exp 2B (Batch Size:128)         97.63      1 min 21.5 seg
3   Exp 2C (Learning Rate: 0.005)         94.94      1 min 30.6 seg
4  Exp 2D (Learning Rate: 0.0005)         97.77      1 min 29.2 seg
5              Exp 2E (Epochs:15)         97.57      2 min 18.2 seg
6               Exp 2F (Epochs:5)         97.03      0 min 46.3 seg


<i> Como conclusión del segundo experimento, podemos señalar lo siguiente:
- Cuando reducimos el tamaño del lote a la mitad ha resultado en una ligera disminución en la precisión y un aumento en el tiempo de ejecución, pero cuando aumentamos el tamaño del lote al doble del modelo de entrenamiento original  mejora un poco el rendimiento y disminuye el tiempo de ejecución lo que podría significar que este modelo se ajusta mejor
- Cuando quintuplicamos la tasa de aprendizaje se reduce notablemente el rendimiento tiende ligeramente a tardar mas que el modelo de entrenamiento original, y si disminuimos a la mitad la tasa de aprendizaje con respecto all original, mejora ligeramente la precisión y tarda casi un segundo más que el modelo original.
- Cuando aumentamos el número de épocas a 15 no se advierte variabilidad en la precisión pero tarda casi uun minuto mas en el tiempo de ejecución, pero cuando se disminuye a 5 épocas tiene menos precision que el original y tarda un poco mas de de la mitad de la duración del modelo de entrenamiento original
- En conclusión final, la configuración con un tamaño de lote de 128, una tasa de aprendizaje de 0.0005 y 10 épocas parece ofrecer el mejor equilibrio entre rendimiento y tiempo de ejecución.

---

### 2.2	Redes convolucionales

#### 2.2.1	Modificaciones de la arquitectura 

El objetivo de esta sección de la actividad es evaluar diversas arquitecturas para el perceptrón multicapa y seleccionar el modelo que obtenga el mejor rendimiento en términos de accuracy. Para ello, se propone modificar los siguientes elementos: el número de filtros, el número de neuronas en la capa posterior a las capas convolucionales, agregando más capas lineales y el parámetro asociado a dropout

##### <i> 1) Probaremos el código orginal del modelo de Redes Convolucionales para tener un parámetro de comparación

In [7]:
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

########################################################################
##########################        DATOS       ##########################
########################################################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

########################################################################
#######################      ARQUITECTURA       ########################
########################################################################

class CNN(nn.Module):
    def __init__(self, verbose=False, filters_l1=32, filters_l2=64, dropout=0.2, final_layer_size=128):
        super(CNN, self).__init__()
        self.verbose = verbose
        self.filters_l1 = filters_l1
        self.filters_l2 = filters_l2
        self.dropout_rate = dropout
        self.final_layer_size = final_layer_size

        self.conv1 = nn.Conv2d(1, self.filters_l1, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(self.filters_l1, self.filters_l2, kernel_size=3, stride=1, padding=1)

        self.fc1_input_size = self._calculate_fc1_input_size()
        
        self.fc1 = nn.Linear(self.fc1_input_size, self.final_layer_size)
        self.dropout = nn.Dropout(self.dropout_rate)
        self.fc2 = nn.Linear(self.final_layer_size, 10)

    def _calculate_fc1_input_size(self):
        with torch.no_grad():
            x = torch.randn(1, 1, 28, 28)
            x = self.pool(torch.relu(self.conv1(x)))
            x = self.pool(torch.relu(self.conv2(x)))
            fc1_input_size = x.numel()
        return fc1_input_size

    def forward(self, x):
        if self.verbose: 
            print(f"Entrada: {x.shape}")

        x = self.pool(torch.relu(self.conv1(x)))
        if self.verbose:
            print(f"Después de Conv1 y MaxPooling: {x.shape}")

        x = self.pool(torch.relu(self.conv2(x)))
        if self.verbose:
            print(f"Después de Conv2 y MaxPooling: {x.shape}")

        x = x.view(-1, self.fc1_input_size)
        if self.verbose:
            print(f"Después de Aplanamiento: {x.shape}")

        x = torch.relu(self.fc1(x))
        if self.verbose:
            print(f"Después de Fully Connected (fc1): {x.shape}")

        x = self.dropout(x)
        if self.verbose:
            print(f"Después de Dropout: {x.shape}")

        x = self.fc2(x)
        if self.verbose:
            print(f"Después de Fully Connected (fc2): {x.shape}")

        return x

########################################################################
#####################  ENTRENAMIENTO Y EVALUACIÓN  #####################
########################################################################

model = CNN(verbose=False, filters_l1=8, filters_l2=32, dropout=0.2, final_layer_size=128).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# FUNCIÓN DE ENTRENAMIENTO
def train(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    return running_loss / len(loader)

# FUNCIÓN DE EVALUACIÓN
def evaluate(model, loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# BUCLE PRINCIPAL DE ENTRENAMIENTO
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    test_accuracy = evaluate(model, test_loader, device)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

final_accuracy = evaluate(model, test_loader, device)
print(f"Final Test Accuracy: {final_accuracy:.4f}")

Epoch [1/10], Loss: 0.3609, Test Accuracy: 0.9715
Epoch [2/10], Loss: 0.0937, Test Accuracy: 0.9813
Epoch [3/10], Loss: 0.0688, Test Accuracy: 0.9851
Epoch [4/10], Loss: 0.0556, Test Accuracy: 0.9874
Epoch [5/10], Loss: 0.0468, Test Accuracy: 0.9893
Epoch [6/10], Loss: 0.0395, Test Accuracy: 0.9893
Epoch [7/10], Loss: 0.0329, Test Accuracy: 0.9891
Epoch [8/10], Loss: 0.0295, Test Accuracy: 0.9895
Epoch [9/10], Loss: 0.0266, Test Accuracy: 0.9898
Epoch [10/10], Loss: 0.0231, Test Accuracy: 0.9895
Final Test Accuracy: 0.9895


##### <i> 2) Experimento 3A: Modificamos la arquitectura del modelo reduciendo la cantidad de filtros a la mitad

In [17]:
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

########################################################################
##########################        DATOS       ##########################
########################################################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

########################################################################
#######################      ARQUITECTURA       ########################
########################################################################

class CNN_Experimento_3A(nn.Module):
    def __init__(self, verbose=False, FL1_Experimemto_3A=16, FL2_Experimento_3A=32, dropout=0.2, final_layer_size=128):
        super(CNN_Experimento_3A, self).__init__()
        self.verbose = verbose
        self.filters_l1 = FL1_Experimemto_3A
        self.filters_l2 = FL2_Experimento_3A
        self.dropout_rate = dropout
        self.final_layer_size = final_layer_size

        self.conv1 = nn.Conv2d(1, self.filters_l1, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(self.filters_l1, self.filters_l2, kernel_size=3, stride=1, padding=1)

        self.fc1_input_size = self._calculate_fc1_input_size()
        
        self.fc1 = nn.Linear(self.fc1_input_size, self.final_layer_size)
        self.dropout = nn.Dropout(self.dropout_rate)
        self.fc2 = nn.Linear(self.final_layer_size, 10)

    def _calculate_fc1_input_size(self):
        with torch.no_grad():
            x = torch.randn(1, 1, 28, 28)
            x = self.pool(torch.relu(self.conv1(x)))
            x = self.pool(torch.relu(self.conv2(x)))
            fc1_input_size = x.numel()
        return fc1_input_size

    def forward(self, x):
        if self.verbose: 
            print(f"Entrada: {x.shape}")

        x = self.pool(torch.relu(self.conv1(x)))
        if self.verbose:
            print(f"Después de Conv1 y MaxPooling: {x.shape}")

        x = self.pool(torch.relu(self.conv2(x)))
        if self.verbose:
            print(f"Después de Conv2 y MaxPooling: {x.shape}")

        x = x.view(-1, self.fc1_input_size)
        if self.verbose:
            print(f"Después de Aplanamiento: {x.shape}")

        x = torch.relu(self.fc1(x))
        if self.verbose:
            print(f"Después de Fully Connected (fc1): {x.shape}")

        x = self.dropout(x)
        if self.verbose:
            print(f"Después de Dropout: {x.shape}")

        x = self.fc2(x)
        if self.verbose:
            print(f"Después de Fully Connected (fc2): {x.shape}")

        return x

########################################################################
#####################  ENTRENAMIENTO Y EVALUACIÓN  #####################
########################################################################

model = CNN_Experimento_3A(verbose=False, FL1_Experimemto_3A=4, FL2_Experimento_3A=16, dropout=0.2, final_layer_size=128).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# FUNCIÓN DE ENTRENAMIENTO
def train(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    return running_loss / len(loader)

# FUNCIÓN DE EVALUACIÓN
def evaluate(model, loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# BUCLE PRINCIPAL DE ENTRENAMIENTO
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    test_accuracy = evaluate(model, test_loader, device)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

final_accuracy = evaluate(model, test_loader, device)
print(f"Final Test Accuracy: {final_accuracy:.4f}")

Epoch [1/10], Loss: 0.3818, Test Accuracy: 0.9681
Epoch [2/10], Loss: 0.1049, Test Accuracy: 0.9774
Epoch [3/10], Loss: 0.0745, Test Accuracy: 0.9829
Epoch [4/10], Loss: 0.0584, Test Accuracy: 0.9851
Epoch [5/10], Loss: 0.0491, Test Accuracy: 0.9865
Epoch [6/10], Loss: 0.0416, Test Accuracy: 0.9872
Epoch [7/10], Loss: 0.0372, Test Accuracy: 0.9884
Epoch [8/10], Loss: 0.0335, Test Accuracy: 0.9889
Epoch [9/10], Loss: 0.0278, Test Accuracy: 0.9890
Epoch [10/10], Loss: 0.0254, Test Accuracy: 0.9884
Final Test Accuracy: 0.9884


##### <i> 3) Experimento 3B: Modificamos la arquitectura del modelo duplicando la cantidad de filtros con respecto al modelo original

In [18]:
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

########################################################################
##########################        DATOS       ##########################
########################################################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

########################################################################
#######################      ARQUITECTURA       ########################
########################################################################

class CNN_Experimento_3A(nn.Module):
    def __init__(self, verbose=False, FL1_Experimemto_3A=64, FL2_Experimento_3A=128, dropout=0.2, final_layer_size=128):
        super(CNN_Experimento_3A, self).__init__()
        self.verbose = verbose
        self.filters_l1 = FL1_Experimemto_3A
        self.filters_l2 = FL2_Experimento_3A
        self.dropout_rate = dropout
        self.final_layer_size = final_layer_size

        self.conv1 = nn.Conv2d(1, self.filters_l1, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(self.filters_l1, self.filters_l2, kernel_size=3, stride=1, padding=1)

        self.fc1_input_size = self._calculate_fc1_input_size()
        
        self.fc1 = nn.Linear(self.fc1_input_size, self.final_layer_size)
        self.dropout = nn.Dropout(self.dropout_rate)
        self.fc2 = nn.Linear(self.final_layer_size, 10)

    def _calculate_fc1_input_size(self):
        with torch.no_grad():
            x = torch.randn(1, 1, 28, 28)
            x = self.pool(torch.relu(self.conv1(x)))
            x = self.pool(torch.relu(self.conv2(x)))
            fc1_input_size = x.numel()
        return fc1_input_size

    def forward(self, x):
        if self.verbose: 
            print(f"Entrada: {x.shape}")

        x = self.pool(torch.relu(self.conv1(x)))
        if self.verbose:
            print(f"Después de Conv1 y MaxPooling: {x.shape}")

        x = self.pool(torch.relu(self.conv2(x)))
        if self.verbose:
            print(f"Después de Conv2 y MaxPooling: {x.shape}")

        x = x.view(-1, self.fc1_input_size)
        if self.verbose:
            print(f"Después de Aplanamiento: {x.shape}")

        x = torch.relu(self.fc1(x))
        if self.verbose:
            print(f"Después de Fully Connected (fc1): {x.shape}")

        x = self.dropout(x)
        if self.verbose:
            print(f"Después de Dropout: {x.shape}")

        x = self.fc2(x)
        if self.verbose:
            print(f"Después de Fully Connected (fc2): {x.shape}")

        return x

########################################################################
#####################  ENTRENAMIENTO Y EVALUACIÓN  #####################
########################################################################

model = CNN_Experimento_3A(verbose=False, FL1_Experimemto_3A=16, FL2_Experimento_3A=64, dropout=0.2, final_layer_size=128).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# FUNCIÓN DE ENTRENAMIENTO
def train(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    return running_loss / len(loader)

# FUNCIÓN DE EVALUACIÓN
def evaluate(model, loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# BUCLE PRINCIPAL DE ENTRENAMIENTO
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    test_accuracy = evaluate(model, test_loader, device)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

final_accuracy = evaluate(model, test_loader, device)
print(f"Final Test Accuracy: {final_accuracy:.4f}")

Epoch [1/10], Loss: 0.2802, Test Accuracy: 0.9802
Epoch [2/10], Loss: 0.0703, Test Accuracy: 0.9866
Epoch [3/10], Loss: 0.0514, Test Accuracy: 0.9888
Epoch [4/10], Loss: 0.0428, Test Accuracy: 0.9890
Epoch [5/10], Loss: 0.0344, Test Accuracy: 0.9916
Epoch [6/10], Loss: 0.0272, Test Accuracy: 0.9915
Epoch [7/10], Loss: 0.0239, Test Accuracy: 0.9900
Epoch [8/10], Loss: 0.0227, Test Accuracy: 0.9915
Epoch [9/10], Loss: 0.0175, Test Accuracy: 0.9923
Epoch [10/10], Loss: 0.0162, Test Accuracy: 0.9921
Final Test Accuracy: 0.9921


##### <i> 4) Experimento 3C: Modificamos la arquitectura del modelo disminuyendo el número de neuronas de la capa posterior a las capas convolucionales a la mitad

In [15]:
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

########################################################################
##########################        DATOS       ##########################
########################################################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

########################################################################
#######################      ARQUITECTURA       ########################
########################################################################

class CNN_Experimento_3C(nn.Module):
    def __init__(self, verbose=False, filters_l1=32, filters_l2=64, dropout=0.2, FLS_Experimento_3C=64):
        super(CNN_Experimento_3C, self).__init__()
        self.verbose = verbose
        self.filters_l1 = filters_l1
        self.filters_l2 = filters_l2
        self.dropout_rate = dropout
        self.final_layer_size = FLS_Experimento_3C

        self.conv1 = nn.Conv2d(1, self.filters_l1, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(self.filters_l1, self.filters_l2, kernel_size=3, stride=1, padding=1)

        self.fc1_input_size = self._calculate_fc1_input_size()
        
        self.fc1 = nn.Linear(self.fc1_input_size, self.final_layer_size)
        self.dropout = nn.Dropout(self.dropout_rate)
        self.fc2 = nn.Linear(self.final_layer_size, 10)

    def _calculate_fc1_input_size(self):
        with torch.no_grad():
            x = torch.randn(1, 1, 28, 28)
            x = self.pool(torch.relu(self.conv1(x)))
            x = self.pool(torch.relu(self.conv2(x)))
            fc1_input_size = x.numel()
        return fc1_input_size

    def forward(self, x):
        if self.verbose: 
            print(f"Entrada: {x.shape}")

        x = self.pool(torch.relu(self.conv1(x)))
        if self.verbose:
            print(f"Después de Conv1 y MaxPooling: {x.shape}")

        x = self.pool(torch.relu(self.conv2(x)))
        if self.verbose:
            print(f"Después de Conv2 y MaxPooling: {x.shape}")

        x = x.view(-1, self.fc1_input_size)
        if self.verbose:
            print(f"Después de Aplanamiento: {x.shape}")

        x = torch.relu(self.fc1(x))
        if self.verbose:
            print(f"Después de Fully Connected (fc1): {x.shape}")

        x = self.dropout(x)
        if self.verbose:
            print(f"Después de Dropout: {x.shape}")

        x = self.fc2(x)
        if self.verbose:
            print(f"Después de Fully Connected (fc2): {x.shape}")

        return x

########################################################################
#####################  ENTRENAMIENTO Y EVALUACIÓN  #####################
########################################################################

model = CNN_Experimento_3A(verbose=False, FL1_Experimemto_3A=32, FL2_Experimento_3A=64, dropout=0.2, final_layer_size=64).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# FUNCIÓN DE ENTRENAMIENTO
def train(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    return running_loss / len(loader)

# FUNCIÓN DE EVALUACIÓN
def evaluate(model, loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# BUCLE PRINCIPAL DE ENTRENAMIENTO
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    test_accuracy = evaluate(model, test_loader, device)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

final_accuracy = evaluate(model, test_loader, device)
print(f"Final Test Accuracy: {final_accuracy:.4f}")

Epoch [1/10], Loss: 0.4129, Test Accuracy: 0.9695
Epoch [2/10], Loss: 0.1183, Test Accuracy: 0.9810
Epoch [3/10], Loss: 0.0844, Test Accuracy: 0.9848
Epoch [4/10], Loss: 0.0708, Test Accuracy: 0.9849
Epoch [5/10], Loss: 0.0610, Test Accuracy: 0.9878
Epoch [6/10], Loss: 0.0531, Test Accuracy: 0.9868
Epoch [7/10], Loss: 0.0491, Test Accuracy: 0.9887
Epoch [8/10], Loss: 0.0430, Test Accuracy: 0.9890
Epoch [9/10], Loss: 0.0379, Test Accuracy: 0.9902
Epoch [10/10], Loss: 0.0358, Test Accuracy: 0.9899
Final Test Accuracy: 0.9899


##### <i> 5) Experimento 3D: Modificamos la arquitectura del modelo aumentando el número de neuronas de la capa posterior a las capas convolucionales al doble

In [19]:
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

########################################################################
##########################        DATOS       ##########################
########################################################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

########################################################################
#######################      ARQUITECTURA       ########################
########################################################################

class CNN_Experimento_3D(nn.Module):
    def __init__(self, verbose=False, filters_l1=32, filters_l2=64, dropout=0.2, FLS_Experimento_3D=256):
        super(CNN_Experimento_3D, self).__init__()
        self.verbose = verbose
        self.filters_l1 = filters_l1
        self.filters_l2 = filters_l2
        self.dropout_rate = dropout
        self.final_layer_size = FLS_Experimento_3D

        self.conv1 = nn.Conv2d(1, self.filters_l1, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(self.filters_l1, self.filters_l2, kernel_size=3, stride=1, padding=1)

        self.fc1_input_size = self._calculate_fc1_input_size()
        
        self.fc1 = nn.Linear(self.fc1_input_size, self.final_layer_size)
        self.dropout = nn.Dropout(self.dropout_rate)
        self.fc2 = nn.Linear(self.final_layer_size, 10)

    def _calculate_fc1_input_size(self):
        with torch.no_grad():
            x = torch.randn(1, 1, 28, 28)
            x = self.pool(torch.relu(self.conv1(x)))
            x = self.pool(torch.relu(self.conv2(x)))
            fc1_input_size = x.numel()
        return fc1_input_size

    def forward(self, x):
        if self.verbose: 
            print(f"Entrada: {x.shape}")

        x = self.pool(torch.relu(self.conv1(x)))
        if self.verbose:
            print(f"Después de Conv1 y MaxPooling: {x.shape}")

        x = self.pool(torch.relu(self.conv2(x)))
        if self.verbose:
            print(f"Después de Conv2 y MaxPooling: {x.shape}")

        x = x.view(-1, self.fc1_input_size)
        if self.verbose:
            print(f"Después de Aplanamiento: {x.shape}")

        x = torch.relu(self.fc1(x))
        if self.verbose:
            print(f"Después de Fully Connected (fc1): {x.shape}")

        x = self.dropout(x)
        if self.verbose:
            print(f"Después de Dropout: {x.shape}")

        x = self.fc2(x)
        if self.verbose:
            print(f"Después de Fully Connected (fc2): {x.shape}")

        return x

########################################################################
#####################  ENTRENAMIENTO Y EVALUACIÓN  #####################
########################################################################

model = CNN_Experimento_3A(verbose=False, FL1_Experimemto_3A=8, FL2_Experimento_3A=32, dropout=0.2, final_layer_size=256).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# FUNCIÓN DE ENTRENAMIENTO
def train(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    return running_loss / len(loader)

# FUNCIÓN DE EVALUACIÓN
def evaluate(model, loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# BUCLE PRINCIPAL DE ENTRENAMIENTO
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    test_accuracy = evaluate(model, test_loader, device)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

final_accuracy = evaluate(model, test_loader, device)
print(f"Final Test Accuracy: {final_accuracy:.4f}")

Epoch [1/10], Loss: 0.2997, Test Accuracy: 0.9767
Epoch [2/10], Loss: 0.0770, Test Accuracy: 0.9842
Epoch [3/10], Loss: 0.0554, Test Accuracy: 0.9842
Epoch [4/10], Loss: 0.0439, Test Accuracy: 0.9874
Epoch [5/10], Loss: 0.0341, Test Accuracy: 0.9895
Epoch [6/10], Loss: 0.0290, Test Accuracy: 0.9897
Epoch [7/10], Loss: 0.0228, Test Accuracy: 0.9919
Epoch [8/10], Loss: 0.0213, Test Accuracy: 0.9910
Epoch [9/10], Loss: 0.0177, Test Accuracy: 0.9921
Epoch [10/10], Loss: 0.0139, Test Accuracy: 0.9920
Final Test Accuracy: 0.9920


##### <i> 5) Experimento 3E: Modificamos la arquitectura del modelo disminuyendo la tasa de Dropout a 0,1

In [20]:

import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

########################################################################
##########################        DATOS       ##########################
########################################################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

########################################################################
#######################      ARQUITECTURA       ########################
########################################################################

class CNN_Experimento_3E(nn.Module):
    def __init__(self, verbose=False, filters_l1=32, filters_l2=64, DP_Experimento_3E=0.1, final_layer_size=128):
        super(CNN_Experimento_3E, self).__init__()
        self.verbose = verbose
        self.filters_l1 = filters_l1
        self.filters_l2 = filters_l2
        self.dropout_rate = DP_Experimento_3E
        self.final_layer_size = final_layer_size

        self.conv1 = nn.Conv2d(1, self.filters_l1, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(self.filters_l1, self.filters_l2, kernel_size=3, stride=1, padding=1)

        self.fc1_input_size = self._calculate_fc1_input_size()
        
        self.fc1 = nn.Linear(self.fc1_input_size, self.final_layer_size)
        self.dropout = nn.Dropout(self.dropout_rate)
        self.fc2 = nn.Linear(self.final_layer_size, 10)

    def _calculate_fc1_input_size(self):
        with torch.no_grad():
            x = torch.randn(1, 1, 28, 28)
            x = self.pool(torch.relu(self.conv1(x)))
            x = self.pool(torch.relu(self.conv2(x)))
            fc1_input_size = x.numel()
        return fc1_input_size

    def forward(self, x):
        if self.verbose: 
            print(f"Entrada: {x.shape}")

        x = self.pool(torch.relu(self.conv1(x)))
        if self.verbose:
            print(f"Después de Conv1 y MaxPooling: {x.shape}")

        x = self.pool(torch.relu(self.conv2(x)))
        if self.verbose:
            print(f"Después de Conv2 y MaxPooling: {x.shape}")

        x = x.view(-1, self.fc1_input_size)
        if self.verbose:
            print(f"Después de Aplanamiento: {x.shape}")

        x = torch.relu(self.fc1(x))
        if self.verbose:
            print(f"Después de Fully Connected (fc1): {x.shape}")

        x = self.dropout(x)
        if self.verbose:
            print(f"Después de Dropout: {x.shape}")

        x = self.fc2(x)
        if self.verbose:
            print(f"Después de Fully Connected (fc2): {x.shape}")

        return x

########################################################################
#####################  ENTRENAMIENTO Y EVALUACIÓN  #####################
########################################################################

model = CNN_Experimento_3A(verbose=False, FL1_Experimemto_3A=8, FL2_Experimento_3A=32, dropout=0.1, final_layer_size=128).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# FUNCIÓN DE ENTRENAMIENTO
def train(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    return running_loss / len(loader)

# FUNCIÓN DE EVALUACIÓN
def evaluate(model, loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# BUCLE PRINCIPAL DE ENTRENAMIENTO
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    test_accuracy = evaluate(model, test_loader, device)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

final_accuracy = evaluate(model, test_loader, device)
print(f"Final Test Accuracy: {final_accuracy:.4f}")

Epoch [1/10], Loss: 0.3413, Test Accuracy: 0.9697
Epoch [2/10], Loss: 0.0822, Test Accuracy: 0.9836
Epoch [3/10], Loss: 0.0572, Test Accuracy: 0.9846
Epoch [4/10], Loss: 0.0428, Test Accuracy: 0.9863
Epoch [5/10], Loss: 0.0344, Test Accuracy: 0.9887
Epoch [6/10], Loss: 0.0294, Test Accuracy: 0.9899
Epoch [7/10], Loss: 0.0249, Test Accuracy: 0.9895
Epoch [8/10], Loss: 0.0203, Test Accuracy: 0.9896
Epoch [9/10], Loss: 0.0178, Test Accuracy: 0.9897
Epoch [10/10], Loss: 0.0154, Test Accuracy: 0.9897
Final Test Accuracy: 0.9897


##### <i> 5) Experimento 3F: Modificamos la arquitectura del modelo incrementando la tasa de Dropout a 0,5

In [21]:

import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

########################################################################
##########################        DATOS       ##########################
########################################################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

########################################################################
#######################      ARQUITECTURA       ########################
########################################################################

class CNN_Experimento_3F(nn.Module):
    def __init__(self, verbose=False, filters_l1=32, filters_l2=64, DP_Experimento_3F=0.5, final_layer_size=128):
        super(CNN_Experimento_3F, self).__init__()
        self.verbose = verbose
        self.filters_l1 = filters_l1
        self.filters_l2 = filters_l2
        self.dropout_rate = DP_Experimento_3F
        self.final_layer_size = final_layer_size

        self.conv1 = nn.Conv2d(1, self.filters_l1, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(self.filters_l1, self.filters_l2, kernel_size=3, stride=1, padding=1)

        self.fc1_input_size = self._calculate_fc1_input_size()
        
        self.fc1 = nn.Linear(self.fc1_input_size, self.final_layer_size)
        self.dropout = nn.Dropout(self.dropout_rate)
        self.fc2 = nn.Linear(self.final_layer_size, 10)

    def _calculate_fc1_input_size(self):
        with torch.no_grad():
            x = torch.randn(1, 1, 28, 28)
            x = self.pool(torch.relu(self.conv1(x)))
            x = self.pool(torch.relu(self.conv2(x)))
            fc1_input_size = x.numel()
        return fc1_input_size

    def forward(self, x):
        if self.verbose: 
            print(f"Entrada: {x.shape}")

        x = self.pool(torch.relu(self.conv1(x)))
        if self.verbose:
            print(f"Después de Conv1 y MaxPooling: {x.shape}")

        x = self.pool(torch.relu(self.conv2(x)))
        if self.verbose:
            print(f"Después de Conv2 y MaxPooling: {x.shape}")

        x = x.view(-1, self.fc1_input_size)
        if self.verbose:
            print(f"Después de Aplanamiento: {x.shape}")

        x = torch.relu(self.fc1(x))
        if self.verbose:
            print(f"Después de Fully Connected (fc1): {x.shape}")

        x = self.dropout(x)
        if self.verbose:
            print(f"Después de Dropout: {x.shape}")

        x = self.fc2(x)
        if self.verbose:
            print(f"Después de Fully Connected (fc2): {x.shape}")

        return x

########################################################################
#####################  ENTRENAMIENTO Y EVALUACIÓN  #####################
########################################################################

model = CNN_Experimento_3A(verbose=False, FL1_Experimemto_3A=8, FL2_Experimento_3A=32, dropout=0.5, final_layer_size=128).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# FUNCIÓN DE ENTRENAMIENTO
def train(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    return running_loss / len(loader)

# FUNCIÓN DE EVALUACIÓN
def evaluate(model, loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# BUCLE PRINCIPAL DE ENTRENAMIENTO
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    test_accuracy = evaluate(model, test_loader, device)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

final_accuracy = evaluate(model, test_loader, device)
print(f"Final Test Accuracy: {final_accuracy:.4f}")

Epoch [1/10], Loss: 0.4097, Test Accuracy: 0.9768
Epoch [2/10], Loss: 0.1272, Test Accuracy: 0.9833
Epoch [3/10], Loss: 0.0937, Test Accuracy: 0.9854
Epoch [4/10], Loss: 0.0801, Test Accuracy: 0.9874
Epoch [5/10], Loss: 0.0653, Test Accuracy: 0.9881
Epoch [6/10], Loss: 0.0603, Test Accuracy: 0.9878
Epoch [7/10], Loss: 0.0537, Test Accuracy: 0.9895
Epoch [8/10], Loss: 0.0479, Test Accuracy: 0.9900
Epoch [9/10], Loss: 0.0423, Test Accuracy: 0.9899
Epoch [10/10], Loss: 0.0401, Test Accuracy: 0.9897
Final Test Accuracy: 0.9897


 Documente los resultados obtenidos, seleccione una arquitecture y entregue sus conclusiones.

In [26]:
import pandas as pd

final_data = {
    "Arquitectura": ["Original", "Experimento 3A", "Experimento 3B", "Experimento 3C", "Experimento 3D", "Experimento 3E", "Experimento 3F"],
    "Final Test Accuracy": [0.9895, 0.9884, 0.9921, 0.9899, 0.9920, 0.9897, 0.9897],
    "Tiempo de ejecución": ["3 min 43.8 seg", "3 min 4.9 seg", "3 min 15.5 seg", "3 min 19.1 seg", "3 min 21.2 seg", "3 min 16.6 seg", "3 min 24.4 seg"]
}

final_df = pd.DataFrame(final_data)

print(final_df)



     Arquitectura  Final Test Accuracy Tiempo de ejecución
0        Original               0.9895      3 min 43.8 seg
1  Experimento 3A               0.9884       3 min 4.9 seg
2  Experimento 3B               0.9921      3 min 15.5 seg
3  Experimento 3C               0.9899      3 min 19.1 seg
4  Experimento 3D               0.9920      3 min 21.2 seg
5  Experimento 3E               0.9897      3 min 16.6 seg
6  Experimento 3F               0.9897      3 min 24.4 seg


<i> Para esta ocasión, modificamos los parámetros de la arquitectura del modelo de redes conovolucionales. Como punto de inicio tenemos la arquitectura original del modelo CNN y nos arroja una precisión de 0,9895 con un tiempo de ejecución de 3 minutos y 43,8 segundos. 
- Como primera prueba, reducimos la cantidad de filtros en las capas conovolucionales a la mitad y resultó que se redujo ligeramente la accuracy final a 0.9884 así como también se redujo el tiempo de ejecución a 3 minutos y 4,9 segundos, resultando el experimento más rápido de esta etapa. En cambio, cuando aumentamos al doble la cantidad filtros en las capas conovolucionales con respecto a la arquitectura original, la precisión aumenta a 0.9921, y el tiempo de ejecución se redujo a a 3 min 15.5 seg, mostrando que un mayor número de filtros no necesariamente significa tiempos de entrenamiento más largos.
- En el experimento 3C reducimos la cantidad de neuronas en la capa final y resulta que la precisión es muy  similar con respecto a la arquitectura original, pero se redujo el tiempo de ejecución en 24,7 segundos menos que la ejecución de la arquitectura original. En el experimento 3D incrementamos al doble las neuronas en la capa final, como resultado mejoró el rendimiento a 0.9920. y el tiempo de ejecución se redujo a 3 minutos y 21,2 segundos, lo que significa que este cambio resultó ser bastante eficiente considerando la mejora en la precisión.
- Reducir el Dropout, logró una precisión de 0,9897 y el tiempo a 3 minutos y 16,6 segundos y si aumentamos el Dropout a 0,5 mantiene la misma precisión con respecto al experimento con Dropout de 0,1, mientras que el tiempo de ejecución fue de 3 minutos y 24.4 segundos.

En conclusión, las configuraciones con un mayor número de filtros y un mayor tamaño de la capa final (Experimentos 3B y 3D) obtuvieron las mejores precisiones finales. Por el lado de la eficiencia de tiempo, la reducción de los filtros y el tamaño de la capa final redujeron el tiempo de ejecución (Experimento 3A), pero también resultaron en una ligera disminución de la precisión. El cambio de valor de dropout a 0.1 o 0.5 no resultó en mejoras significativas en la precisión final, pero tuvo un impacto en el tiempo de ejecución.

Finalmente, la configuración del experimento 3B con 32/128 filtros, tasa de dropout de 0,2 y capa final de 128 neuronas parece ofrecer el mejor equilibrio entre precisión y tiempo de ejecución.


---

##### 2.2.2	Modificaciones del entrenamiento

Al igual que la experimentación realizada con el perceptrón multicapa, compare diferentes algoritmos de optimización para el entrenamiento y evalúe el comportamiento del entrenamiento para diferentes valores del learning rate.

##### <i> 1) Probaremos el código orginal del modelo de Redes Convolucionales para tener un parámetro de comparación

In [27]:
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

########################################################################
##########################        DATOS       ##########################
########################################################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

########################################################################
#######################      ARQUITECTURA       ########################
########################################################################

class CNN(nn.Module):
    def __init__(self, verbose=False, filters_l1=32, filters_l2=64, dropout=0.2, final_layer_size=128):
        super(CNN, self).__init__()
        self.verbose = verbose
        self.filters_l1 = filters_l1
        self.filters_l2 = filters_l2
        self.dropout_rate = dropout
        self.final_layer_size = final_layer_size

        self.conv1 = nn.Conv2d(1, self.filters_l1, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(self.filters_l1, self.filters_l2, kernel_size=3, stride=1, padding=1)

        self.fc1_input_size = self._calculate_fc1_input_size()
        
        self.fc1 = nn.Linear(self.fc1_input_size, self.final_layer_size)
        self.dropout = nn.Dropout(self.dropout_rate)
        self.fc2 = nn.Linear(self.final_layer_size, 10)

    def _calculate_fc1_input_size(self):
        with torch.no_grad():
            x = torch.randn(1, 1, 28, 28)
            x = self.pool(torch.relu(self.conv1(x)))
            x = self.pool(torch.relu(self.conv2(x)))
            fc1_input_size = x.numel()
        return fc1_input_size

    def forward(self, x):
        if self.verbose: 
            print(f"Entrada: {x.shape}")

        x = self.pool(torch.relu(self.conv1(x)))
        if self.verbose:
            print(f"Después de Conv1 y MaxPooling: {x.shape}")

        x = self.pool(torch.relu(self.conv2(x)))
        if self.verbose:
            print(f"Después de Conv2 y MaxPooling: {x.shape}")

        x = x.view(-1, self.fc1_input_size)
        if self.verbose:
            print(f"Después de Aplanamiento: {x.shape}")

        x = torch.relu(self.fc1(x))
        if self.verbose:
            print(f"Después de Fully Connected (fc1): {x.shape}")

        x = self.dropout(x)
        if self.verbose:
            print(f"Después de Dropout: {x.shape}")

        x = self.fc2(x)
        if self.verbose:
            print(f"Después de Fully Connected (fc2): {x.shape}")

        return x

########################################################################
#####################  ENTRENAMIENTO Y EVALUACIÓN  #####################
########################################################################

model = CNN(verbose=False, filters_l1=8, filters_l2=32, dropout=0.2, final_layer_size=128).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# FUNCIÓN DE ENTRENAMIENTO
def train(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    return running_loss / len(loader)

# FUNCIÓN DE EVALUACIÓN
def evaluate(model, loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# BUCLE PRINCIPAL DE ENTRENAMIENTO
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    test_accuracy = evaluate(model, test_loader, device)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

final_accuracy = evaluate(model, test_loader, device)
print(f"Final Test Accuracy: {final_accuracy:.4f}")

Epoch [1/10], Loss: 0.3220, Test Accuracy: 0.9776
Epoch [2/10], Loss: 0.0803, Test Accuracy: 0.9843
Epoch [3/10], Loss: 0.0560, Test Accuracy: 0.9879
Epoch [4/10], Loss: 0.0454, Test Accuracy: 0.9895
Epoch [5/10], Loss: 0.0371, Test Accuracy: 0.9900
Epoch [6/10], Loss: 0.0306, Test Accuracy: 0.9903
Epoch [7/10], Loss: 0.0268, Test Accuracy: 0.9922
Epoch [8/10], Loss: 0.0232, Test Accuracy: 0.9897
Epoch [9/10], Loss: 0.0206, Test Accuracy: 0.9889
Epoch [10/10], Loss: 0.0170, Test Accuracy: 0.9913
Final Test Accuracy: 0.9913


##### <i> 2) Experimento 4A: Reducimos a la mitad tasa de aprendizaje del modelo del optimizador Adam para el entrenamiento del modelo

In [29]:
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

########################################################################
##########################        DATOS       ##########################
########################################################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

########################################################################
#######################      ARQUITECTURA       ########################
########################################################################

class CNN(nn.Module):
    def __init__(self, verbose=False, filters_l1=32, filters_l2=64, dropout=0.2, final_layer_size=128):
        super(CNN, self).__init__()
        self.verbose = verbose
        self.filters_l1 = filters_l1
        self.filters_l2 = filters_l2
        self.dropout_rate = dropout
        self.final_layer_size = final_layer_size

        self.conv1 = nn.Conv2d(1, self.filters_l1, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(self.filters_l1, self.filters_l2, kernel_size=3, stride=1, padding=1)

        self.fc1_input_size = self._calculate_fc1_input_size()
        
        self.fc1 = nn.Linear(self.fc1_input_size, self.final_layer_size)
        self.dropout = nn.Dropout(self.dropout_rate)
        self.fc2 = nn.Linear(self.final_layer_size, 10)

    def _calculate_fc1_input_size(self):
        with torch.no_grad():
            x = torch.randn(1, 1, 28, 28)
            x = self.pool(torch.relu(self.conv1(x)))
            x = self.pool(torch.relu(self.conv2(x)))
            fc1_input_size = x.numel()
        return fc1_input_size

    def forward(self, x):
        if self.verbose: 
            print(f"Entrada: {x.shape}")

        x = self.pool(torch.relu(self.conv1(x)))
        if self.verbose:
            print(f"Después de Conv1 y MaxPooling: {x.shape}")

        x = self.pool(torch.relu(self.conv2(x)))
        if self.verbose:
            print(f"Después de Conv2 y MaxPooling: {x.shape}")

        x = x.view(-1, self.fc1_input_size)
        if self.verbose:
            print(f"Después de Aplanamiento: {x.shape}")

        x = torch.relu(self.fc1(x))
        if self.verbose:
            print(f"Después de Fully Connected (fc1): {x.shape}")

        x = self.dropout(x)
        if self.verbose:
            print(f"Después de Dropout: {x.shape}")

        x = self.fc2(x)
        if self.verbose:
            print(f"Después de Fully Connected (fc2): {x.shape}")

        return x

########################################################################
#####################  ENTRENAMIENTO Y EVALUACIÓN  #####################
########################################################################

model = CNN(verbose=False, filters_l1=8, filters_l2=32, dropout=0.2, final_layer_size=128).to(device)
criterion = nn.CrossEntropyLoss()
Experimento_4A = optim.Adam(model.parameters(), lr=0.0005)

# FUNCIÓN DE ENTRENAMIENTO
def train(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    return running_loss / len(loader)

# FUNCIÓN DE EVALUACIÓN
def evaluate(model, loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# BUCLE PRINCIPAL DE ENTRENAMIENTO
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, Experimento_4A, device)
    test_accuracy = evaluate(model, test_loader, device)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

final_accuracy = evaluate(model, test_loader, device)
print(f"Final Test Accuracy: {final_accuracy:.4f}")

Epoch [1/10], Loss: 0.5147, Test Accuracy: 0.9598
Epoch [2/10], Loss: 0.1347, Test Accuracy: 0.9749
Epoch [3/10], Loss: 0.0939, Test Accuracy: 0.9814
Epoch [4/10], Loss: 0.0745, Test Accuracy: 0.9823
Epoch [5/10], Loss: 0.0631, Test Accuracy: 0.9850
Epoch [6/10], Loss: 0.0534, Test Accuracy: 0.9855
Epoch [7/10], Loss: 0.0473, Test Accuracy: 0.9857
Epoch [8/10], Loss: 0.0418, Test Accuracy: 0.9869
Epoch [9/10], Loss: 0.0376, Test Accuracy: 0.9888
Epoch [10/10], Loss: 0.0333, Test Accuracy: 0.9884
Final Test Accuracy: 0.9884


##### <i> 2) Experimento 4B: Aumentamos al doble la tasa de aprendizaje del modelo original del optimizador Adam para el entrenamiento del modelo

In [31]:
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

########################################################################
##########################        DATOS       ##########################
########################################################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

########################################################################
#######################      ARQUITECTURA       ########################
########################################################################

class CNN(nn.Module):
    def __init__(self, verbose=False, filters_l1=32, filters_l2=64, dropout=0.2, final_layer_size=128):
        super(CNN, self).__init__()
        self.verbose = verbose
        self.filters_l1 = filters_l1
        self.filters_l2 = filters_l2
        self.dropout_rate = dropout
        self.final_layer_size = final_layer_size

        self.conv1 = nn.Conv2d(1, self.filters_l1, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(self.filters_l1, self.filters_l2, kernel_size=3, stride=1, padding=1)

        self.fc1_input_size = self._calculate_fc1_input_size()
        
        self.fc1 = nn.Linear(self.fc1_input_size, self.final_layer_size)
        self.dropout = nn.Dropout(self.dropout_rate)
        self.fc2 = nn.Linear(self.final_layer_size, 10)

    def _calculate_fc1_input_size(self):
        with torch.no_grad():
            x = torch.randn(1, 1, 28, 28)
            x = self.pool(torch.relu(self.conv1(x)))
            x = self.pool(torch.relu(self.conv2(x)))
            fc1_input_size = x.numel()
        return fc1_input_size

    def forward(self, x):
        if self.verbose: 
            print(f"Entrada: {x.shape}")

        x = self.pool(torch.relu(self.conv1(x)))
        if self.verbose:
            print(f"Después de Conv1 y MaxPooling: {x.shape}")

        x = self.pool(torch.relu(self.conv2(x)))
        if self.verbose:
            print(f"Después de Conv2 y MaxPooling: {x.shape}")

        x = x.view(-1, self.fc1_input_size)
        if self.verbose:
            print(f"Después de Aplanamiento: {x.shape}")

        x = torch.relu(self.fc1(x))
        if self.verbose:
            print(f"Después de Fully Connected (fc1): {x.shape}")

        x = self.dropout(x)
        if self.verbose:
            print(f"Después de Dropout: {x.shape}")

        x = self.fc2(x)
        if self.verbose:
            print(f"Después de Fully Connected (fc2): {x.shape}")

        return x

########################################################################
#####################  ENTRENAMIENTO Y EVALUACIÓN  #####################
########################################################################

model = CNN(verbose=False, filters_l1=8, filters_l2=32, dropout=0.2, final_layer_size=128).to(device)
criterion = nn.CrossEntropyLoss()
Experimento_4B = optim.Adam(model.parameters(), lr=0.002)

# FUNCIÓN DE ENTRENAMIENTO
def train(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    return running_loss / len(loader)

# FUNCIÓN DE EVALUACIÓN
def evaluate(model, loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# BUCLE PRINCIPAL DE ENTRENAMIENTO
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, Experimento_4B, device)
    test_accuracy = evaluate(model, test_loader, device)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

final_accuracy = evaluate(model, test_loader, device)
print(f"Final Test Accuracy: {final_accuracy:.4f}")

Epoch [1/10], Loss: 0.2500, Test Accuracy: 0.9794
Epoch [2/10], Loss: 0.0685, Test Accuracy: 0.9877
Epoch [3/10], Loss: 0.0486, Test Accuracy: 0.9872
Epoch [4/10], Loss: 0.0394, Test Accuracy: 0.9903
Epoch [5/10], Loss: 0.0316, Test Accuracy: 0.9883
Epoch [6/10], Loss: 0.0278, Test Accuracy: 0.9900
Epoch [7/10], Loss: 0.0250, Test Accuracy: 0.9909
Epoch [8/10], Loss: 0.0217, Test Accuracy: 0.9906
Epoch [9/10], Loss: 0.0203, Test Accuracy: 0.9897
Epoch [10/10], Loss: 0.0166, Test Accuracy: 0.9895
Final Test Accuracy: 0.9895


##### <i> 2) Experimento 4C: Quintuplicamos la tasa de aprendizaje del modelo original del optimizador Adam para el entrenamiento del modelo

In [34]:
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

########################################################################
##########################        DATOS       ##########################
########################################################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

########################################################################
#######################      ARQUITECTURA       ########################
########################################################################

class CNN(nn.Module):
    def __init__(self, verbose=False, filters_l1=32, filters_l2=64, dropout=0.2, final_layer_size=128):
        super(CNN, self).__init__()
        self.verbose = verbose
        self.filters_l1 = filters_l1
        self.filters_l2 = filters_l2
        self.dropout_rate = dropout
        self.final_layer_size = final_layer_size

        self.conv1 = nn.Conv2d(1, self.filters_l1, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(self.filters_l1, self.filters_l2, kernel_size=3, stride=1, padding=1)

        self.fc1_input_size = self._calculate_fc1_input_size()
        
        self.fc1 = nn.Linear(self.fc1_input_size, self.final_layer_size)
        self.dropout = nn.Dropout(self.dropout_rate)
        self.fc2 = nn.Linear(self.final_layer_size, 10)

    def _calculate_fc1_input_size(self):
        with torch.no_grad():
            x = torch.randn(1, 1, 28, 28)
            x = self.pool(torch.relu(self.conv1(x)))
            x = self.pool(torch.relu(self.conv2(x)))
            fc1_input_size = x.numel()
        return fc1_input_size

    def forward(self, x):
        if self.verbose: 
            print(f"Entrada: {x.shape}")

        x = self.pool(torch.relu(self.conv1(x)))
        if self.verbose:
            print(f"Después de Conv1 y MaxPooling: {x.shape}")

        x = self.pool(torch.relu(self.conv2(x)))
        if self.verbose:
            print(f"Después de Conv2 y MaxPooling: {x.shape}")

        x = x.view(-1, self.fc1_input_size)
        if self.verbose:
            print(f"Después de Aplanamiento: {x.shape}")

        x = torch.relu(self.fc1(x))
        if self.verbose:
            print(f"Después de Fully Connected (fc1): {x.shape}")

        x = self.dropout(x)
        if self.verbose:
            print(f"Después de Dropout: {x.shape}")

        x = self.fc2(x)
        if self.verbose:
            print(f"Después de Fully Connected (fc2): {x.shape}")

        return x

########################################################################
#####################  ENTRENAMIENTO Y EVALUACIÓN  #####################
########################################################################

model = CNN(verbose=False, filters_l1=8, filters_l2=32, dropout=0.2, final_layer_size=128).to(device)
criterion = nn.CrossEntropyLoss()
Experimento_4C = optim.Adam(model.parameters(), lr=0.005)

# FUNCIÓN DE ENTRENAMIENTO
def train(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    return running_loss / len(loader)

# FUNCIÓN DE EVALUACIÓN
def evaluate(model, loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# BUCLE PRINCIPAL DE ENTRENAMIENTO
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, Experimento_4C, device)
    test_accuracy = evaluate(model, test_loader, device)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

final_accuracy = evaluate(model, test_loader, device)
print(f"Final Test Accuracy: {final_accuracy:.4f}")

Epoch [1/10], Loss: 0.2604, Test Accuracy: 0.9826
Epoch [2/10], Loss: 0.0673, Test Accuracy: 0.9858
Epoch [3/10], Loss: 0.0558, Test Accuracy: 0.9866
Epoch [4/10], Loss: 0.0469, Test Accuracy: 0.9872
Epoch [5/10], Loss: 0.0414, Test Accuracy: 0.9886
Epoch [6/10], Loss: 0.0359, Test Accuracy: 0.9862
Epoch [7/10], Loss: 0.0342, Test Accuracy: 0.9871
Epoch [8/10], Loss: 0.0309, Test Accuracy: 0.9884
Epoch [9/10], Loss: 0.0287, Test Accuracy: 0.9909
Epoch [10/10], Loss: 0.0272, Test Accuracy: 0.9890
Final Test Accuracy: 0.9890


En esta última etapa experimental modificamos la tasa de aprendizaje del optimizador Adam para el entrenamiento del modelo de redes conovolucionales. Evaluamos el modelo original para tener un parámetro de comparación el cual tiene tasa de aprendizaje original de 0.001 y nos arrojó el resultado del Final Test Accuracy de 0.9913 y un tiempo de ejeución de 3 minutos y 33.8 segundos.
Cuando reducimos a la mitad la tasa de aprendizaje, resultó en una mejora más gradual y una precisión final ligeramente menor (0.9884), con un tiempo de ejecución un poco más largo (3 minutos y 37.2 segundos).
Cuando aumentamos learning rate a 0.002 proporcionó una precisión final  de 0.9895 y un tiempo de ejecución de 3 minutos y 41.8 segundos.
Cuando quituplicamos el learning rate a un 0.005 resultó en una precisión final más baja (0.9890) y el tiempo de ejecución más largo (3 minutos y 46.1 segundos), indicando que puede no ser ideal para este modelo en particular.
Como conclusión, la configuración con un learning rate de 0.001 correspondiente al modelo de optimizador para el entrenamiento original parece ofrecer el mejor equilibrio entre precisión y tiempo de ejecución, haciendo que sea la opción preferida para el entrenamiento de este modelo

---

### 2.3	Comparación

Concluya comparando el rendimiento, el número de parámetros y tiempo de ejecución de cada una de las arquitecturas

<i> En este miniproyecto se ha realizado 18 experimentos, 9 por cada clase de red neuronal, para mejor viasualización se adjuta junto al trabajo el archivo excel que entrega los resultados completos para todoos los experimentos realizados.

Del análisis realizado se puede concluir lo siguiente:

Perceptrón Multicapa (MLP):
- Mejor Modelo: Experimento 2D con Batch Size: 64, Learning Rate: 0.0005, Epochs: 10 obtuvo la mejor precisión de 97.77% y un tiempo de ejecución aceptable de 1 min 29.2 seg.
- Peor Modelo: Experimento 2C con Batch Size: 64, Learning Rate: 0.005, Epochs: 10 tuvo la peor precisión de 94.94%, indicando que un learning rate alto puede perjudicar el rendimiento.

Observaciones: Los cambios en el tamaño del lote y el learning rate pueden tener un impacto significativo en la precisión y el tiempo de ejecución. Incrementar las épocas generalmente mejora la precisión, pero a costa de un mayor tiempo de ejecución.

Redes Neuronales Convolucionales (CNN):
- Mejor Modelo: Experimento 3B con Filtros: 32/128, Dropout: 0.2, Final layer size: 128 obtuvo la mejor precisión de 0.9921 y un tiempo de ejecución razonable de 3 min 15.5 seg.
- Peor Modelo: Experimento 3A con Filtros: 16/32, Dropout: 0.2, Final layer size: 128 tuvo una precisión ligeramente menor de 0.9884, pero fue más rápido con 3 min 4.9 seg.

Observaciones: Ajustar los filtros y la capa final puede mejorar significativamente la precisión. Los cambios en el learning rate también muestran variabilidad en el rendimiento, con una tasa más alta que no siempre produce mejores resultados.

Conclusión final:

MLP vs. CNN: Las Redes Neuronales Convolucionales (CNN) generalmente muestran una mejor precisión comparada con el Perceptrón Multicapa (MLP), especialmente con arquitecturas ajustadas correctamente.

Entrenamiento: Tanto en MLP como en CNN, ajustar los parámetros de entrenamiento como el learning rate, tamaño del lote y número de épocas es crucial para encontrar el mejor rendimiento.

Tiempo de Ejecución: Las CNN tienden a tener un mayor tiempo de ejecución comparado con los MLP, pero esto se compensa con una mayor precisión en la mayoría de los casos.